# Feature Detection and Extraction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.color import rgb2gray
from skimage.transform import rescale
from skimage.feature import corner_harris, corner_peaks, hog
from skimage.util import img_as_float
import imageio.v3 as iio
from io import BytesIO
from pyodide.http import pyfetch

# -------------------------------
# Image reader from URL using pyodide
# -------------------------------
async def read_image_from_url(url: str) -> np.ndarray:
    response = await pyfetch(url=url, method="GET")
    data = BytesIO(await response.bytes())
    img = iio.imread(data, index=None)
    return img

# -------------------------------
# Harris + HOG analysis function
# -------------------------------
async def analyze_image_scale_invariance(image_url: str):
    # Step 1: Read image
    img = await read_image_from_url(image_url)

    # Convert to grayscale
    if img.ndim == 3:
        img_gray = rgb2gray(img)
    else:
        img_gray = img

    # Convert to float for skimage compatibility
    img_gray = img_as_float(img_gray)

    # Step 2: Harris corner detection at original scale
    harris_orig = corner_harris(img_gray, k=0.05)
    corners_orig = corner_peaks(harris_orig, min_distance=5)

    # Step 3: Rescale image (e.g., 0.5x)
    img_small = rescale(img_gray, 0.5, anti_aliasing=True)
    
    print("Original shape:", img_gray.shape)
    print("Scaled-down shape:", img_small.shape)

    harris_small = corner_harris(img_small, k=0.05)
    corners_small = corner_peaks(harris_small, min_distance=5)

    # Step 4: Plot Harris corner results
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    ax[0].imshow(img_gray, cmap='gray')
    ax[0].plot(corners_orig[:, 1], corners_orig[:, 0], 'ro', markersize=3)
    ax[0].set_title('Harris Corners (Original Scale)')
    ax[0].axis('off')

    ax[1].imshow(img_small, cmap='gray')
    ax[1].plot(corners_small[:, 1], corners_small[:, 0], 'ro', markersize=3)
    ax[1].set_title('Harris Corners (Scaled Down 0.5x)')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

    # Step 5: HOG descriptor visualization
    _, hog_img_orig = hog(
        img_gray, orientations=9, pixels_per_cell=(8, 8),
        cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys'
    )

    _, hog_img_small = hog(
        img_small, orientations=9, pixels_per_cell=(8, 8),
        cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys'
    )

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    ax[0].imshow(hog_img_orig, cmap='gray')
    ax[0].set_title("HOG (Original Scale)")
    ax[0].axis('off')

    ax[1].imshow(hog_img_small, cmap='gray')
    ax[1].set_title("HOG (Scaled Down 0.5x)")
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

# -------------------------------
# Example Usage
# -------------------------------
async def main():
    IMAGE_URL = "https://raw.githubusercontent.com/canberkgurel/DisparityMapfromStereoPair/master/tsukuba_l.png"
    await analyze_image_scale_invariance(IMAGE_URL)

# Call main
await main()